In [12]:
from azure.storage.blob import BlobServiceClient, BlobClient, ContainerClient
import openai
from openai import OpenAI
from dotenv import load_dotenv, find_dotenv
import os

In [13]:
blob_service_client = BlobServiceClient.from_connection_string(os.getenv("AZURE_STORAGE_CONNECTION_STRING"))
container_client = blob_service_client.get_container_client(os.getenv("AZURE_CONTAINER"))

client = OpenAI(
    api_key=os.getenv("OPENAI_API_KEY"),
)

In [15]:
blob_paths = [
'BUS5000/Introduction/10dayMBA - Intro.pdf',
'BUS5000/Introduction/STUDENT_NOTES/s1/s1w0.docx',
'BUS5000/Introduction/STUDENT_NOTES/s2/s2w0.docx'
]
# List to store file objects
uploaded_files = []

for blob_path in blob_paths:
    # Download the file from Azure Blob
    blob_client = container_client.get_blob_client(blob_path)
    with open(blob_path, "wb") as download_file:
        download_file.write(blob_client.download_blob().readall())

    # Upload the file to OpenAI
    with open(blob_path, "rb") as file:
        response = client.files.create(file=file, purpose="assistants")
        uploaded_files.append(response)

In [18]:
for file in uploaded_files:
    client.files.delete(file.id)
